Data Preperation

In [1]:
import pandas as pd
import numpy as np
from itertools import chain
from sklearn.decomposition import PCA

In [2]:
training = pd.read_csv('HMAHCC_COMP.csv')

C:\Users\cgn31\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
class DataPrep:
    def __init__(self, df):
        self.raw_df = df
    
    def seperate_frame(self):
        new_diagnosis_col = ['New diagnosis - CPD',
                         'New diagnosis - Hypertension', 
                         'New diagnosis - Top 5', 
                         'New diagnosis - CAD', 
                         'New diagnosis - CHF', 
                         'New diagnosis - Diabetes']

        call_col = ['Inbound Call by Prov',
                    'Inbound Call by Mbr',
                    'Inbound Call by Other']

        non_rx_claim_col = ['Surgery',
                            'Fully Paid Claim']

        rx_claim_col = ['RX Claim - New Drug',
                        'RX Claim - First Time Mail Order']
        
        self.new_diagnosis = self.raw_df[self.raw_df['event_descr'].isin(new_diagnosis_col)]

        self.new_diagnosis = self.new_diagnosis.drop(['event_attr6',
                                                      'event_attr7',
                                                      'event_attr8',
                                                      'event_attr9',
                                                      'event_attr10',
                                                      'PAY_DAY_SUPPLY_CNT',
                                                      'PAYABLE_QTY',
                                                      'MME',
                                                      'DRUG_TYPE',
                                                      'Specialty',
                                                      'Specialty2',
                                                      'Specialty3'],
                                                       axis = 1)

        self.new_diagnosis.columns = ['id',
                                 'event_descr',
                                 'diagnosis',
                                 'place_of_treatment',
                                 'charge_amount',
                                 'net_paid_amount',
                                 'member_responsible_amount',
                                 'Days']
        
        self.call = self.raw_df[self.raw_df['event_descr'].isin(call_col)]

        self.call = self.call.drop(['event_attr6',
                          'event_attr7',
                          'event_attr8',
                          'event_attr9',
                          'event_attr10',
                          'PAY_DAY_SUPPLY_CNT',
                          'PAYABLE_QTY',
                          'MME',
                          'DRUG_TYPE',
                          'Specialty',
                          'Specialty2',
                          'Specialty3'],
                           axis = 1)

        self.call.columns = ['id',
                        'event_descr',
                        'call_category',
                        'inquiry_reason_description',
                        'disposition_description',
                        'origin',
                        'location',
                        'Days']
        
        self.non_rx_claim = self.raw_df[self.raw_df['event_descr'].isin(non_rx_claim_col)]

        self.non_rx_claim = self.non_rx_claim.drop(['event_attr6',
                                          'event_attr7',
                                          'event_attr8',
                                          'event_attr9',
                                          'event_attr10',
                                          'PAY_DAY_SUPPLY_CNT',
                                          'PAYABLE_QTY',
                                          'MME',
                                          'DRUG_TYPE',
                                          'Specialty',
                                          'Specialty2',
                                          'Specialty3'],
                                           axis = 1)

        self.non_rx_claim.columns = ['id',
                                'event_descr',
                                'diagnosis',
                                'place_of_treatment',
                                'charge_amount',
                                'net_paid_amount',
                                'member_responsible_amount',
                                'Days']
        
        self.new_provider = self.raw_df[self.raw_df['event_descr']=='New provider']
        self.new_provider = self.new_provider.iloc[:,:2]
        
        self.rx_claim = self.raw_df[self.raw_df['event_descr'].isin(rx_claim_col)]

        self.rx_claim = self.rx_claim.drop(['event_attr6',
                          'event_attr7'],
                           axis = 1)

        self.rx_claim.columns = ['id',
                            'event_descr',
                            'gpi_drug_group6_id',
                            'gpi_drug_class_description',
                            'brand_name',
                            'drug_group_id',
                            'generic_name',
                            'drug_group_description',
                            'member_responsible_amount',
                            'gpi_drug_group8_id',
                            'Days',
                            'PAY_DAY_SUPPLY_CNT',
                            'PAYABLE_QTY',
                            'MME',
                            'DRUG_TYPE',
                            'Specialty',
                            'Specialty2',
                            'Specialty3']
        
        self.rx_paid = self.raw_df[self.raw_df['event_descr']=='RX Claim - Paid']

        self.rx_paid = self.rx_paid.drop(['event_attr2',
                                'event_attr7'],
                                 axis = 1)

        self.rx_paid.columns = ['id',
                           'event_descr',
                           'gpi_drug_class_description',
                           'rx_cost',
                           'net_paid_amount',
                           'brand_name',
                           'drug_group_description',
                           'generic_name',
                           'member_responsible_amount',
                           'gpi_drug_group8_id',
                           'Days',
                           'PAY_DAY_SUPPLY_CNT',
                           'PAYABLE_QTY',
                           'MME',
                           'DRUG_TYPE',
                           'Specialty',
                           'Specialty2',
                           'Specialty3']
        
        self.rx_reject = self.raw_df[self.raw_df['event_descr']=='RX Claim - Rejected']

        self.rx_reject = self.rx_reject.drop(['PAY_DAY_SUPPLY_CNT',
                                    'PAYABLE_QTY',
                                    'MME',
                                    'DRUG_TYPE',
                                    'Specialty',
                                    'Specialty2',
                                    'Specialty3'],
                                     axis =1)

        self.rx_reject.columns = ['id',
                             'event_descr',
                             'status_code',
                             'diagnosis',
                             'cob',
                             'claim_tier',
                             'brand_name',
                             'generic_name',
                             'ndc_id',
                             'pay_day_supply_count',
                             'member_responsible_amount',
                             'gpi_drug_group8_id',
                             'Days']
        
    def get_opioid_data(self):
        self.opioid_data = self.raw_df[self.raw_df['PAY_DAY_SUPPLY_CNT'].notnull()]
        self.opioid_data_grouped = self.opioid_data.groupby(by='id')
        
    def LTOT(self):
        
        def get_LTOT(ID):
            try:
                group = self.opioid_data_grouped.get_group(ID)
                frame = group[['Days', 'PAY_DAY_SUPPLY_CNT']].drop_duplicates()
                frame = frame[frame['Days']>=0]
                frame['drugs until'] = (frame['Days'] + frame['PAY_DAY_SUPPLY_CNT']).astype(int)
                frame['range'] = frame.apply(lambda x : range(x['Days'].astype(int),x['drugs until'].astype(int)),1)

                concat = concatenated = chain(*list(frame['range']))
                concat = set(concat)

                day_frame = pd.DataFrame(columns = ['Days', 'Has Drug?'])
                day_frame['Days'] = range(max(concat)+1)
                day_frame['Has Drug?'] = (day_frame['Days'].isin(concat))

                for n in range(180,len(day_frame)+1):
                    frame_slice = day_frame.iloc[n-180:n]
                    drug_days = np.sum(frame_slice['Has Drug?'])

                    if drug_days >= 162:
                        return (True, n-180, n)

                return (False, np.nan, np.nan)
            except:
                return (np.nan, np.nan, np.nan)
        
        id_list = self.raw_df['id'].drop_duplicates().values
        
        response_variable = pd.DataFrame(id_list, columns = ['id'])
        response_variable['LTOT'] = response_variable['id'].map(get_LTOT)
        
        response_variable[['LTOT', 'Begining Date', 'End Date']] = \
        pd.DataFrame(response_variable['LTOT'].tolist(), index = response_variable.index)
        
        self.response_variable = response_variable.set_index('id')
    
    
    def main_feature_extraction(self): 
        opioid_all_time = self.rx_paid[self.rx_paid['PAY_DAY_SUPPLY_CNT'].notnull()]['generic_name'].value_counts()
        mask = self.rx_paid['generic_name'].map(lambda x: x in opioid_all_time.index.values)
        true_opioid = self.rx_paid[mask]
        
        opioid_grouped = true_opioid.groupby(by=['id'])

        idtestlist = true_opioid['id'].drop_duplicates()
        features3 = pd.DataFrame()

        for ID in idtestlist:
            tmp = opioid_grouped.get_group(ID)

            # MME (per day) on day 0
            # Suuply_CNT on day 0
            on_day0 = tmp[tmp['Days'] == 0] 
            if not on_day0.empty:
                MME0 = on_day0['MME'].values[0]
                SC0 = on_day0['PAY_DAY_SUPPLY_CNT'].values[0]
                PQ0 = on_day0['PAYABLE_QTY'].values[0]
            else:
                MME0 = 0
                SC0 = 0
                PQ0 = 0

            # max MME (per day) prior to day 0
            # average MME (per day) prior to day 0
            # Total Supply_CNT prior to day 0
            prior_day0 = tmp[tmp['Days'] < 0]
            if not prior_day0.empty:
                maxMME = np.nanmax(prior_day0['MME'].values)
                avgMME = np.nanmean(prior_day0['MME'].values)
                totalSC = np.nansum(prior_day0['PAY_DAY_SUPPLY_CNT'].values)
                totalPQ = np.nansum(prior_day0['PAYABLE_QTY'].values)
            else:
                maxMME = 0
                avgMME = 0
                totalSC = 0
                totalPQ = 0

            output = pd.DataFrame({'MME_on_day0': MME0, 
                                 'SUPPLY_CNT_on_day0': SC0,
                                   'PAYABLE_QTY_on_day0': PQ0,
                                 'max_MME_prior': maxMME,
                                 'avg_MME_prior': avgMME,
                                 'total_SUPPLY_CNT_prior': totalSC,
                                  'total_PAYABLE_QTY_prior': totalPQ},
                                  index = [ID])

            features3 = features3.append(output, sort=False)

        # MME_on_day0, max_MME_prior, avg_MME_prior has some missing value, fill with medians
        features3['MME_on_day0'] = features3['MME_on_day0'].fillna(np.nanmedian(features3['MME_on_day0']))
        features3['max_MME_prior'] = features3['max_MME_prior'].fillna(np.nanmedian(features3['max_MME_prior']))
        features3['avg_MME_prior'] = features3['avg_MME_prior'].fillna(np.nanmedian(features3['avg_MME_prior']))

        # add one more feature: supply_times
        supply_times = true_opioid[true_opioid['Days']<=0].groupby(by=['id'])['PAY_DAY_SUPPLY_CNT'].count()
        supply_times = pd.DataFrame(supply_times)
        supply_times.columns = ['supply_times']
        features3 = features3.merge(supply_times, left_on=features3.index.values, right_on=supply_times.index.values)
        features3 = features3.set_index('key_0')
        
        self.main_features = features3
        
    def generic_feature_extraction(self):
        opioid_all_time = self.rx_paid[self.rx_paid['PAY_DAY_SUPPLY_CNT'].notnull()]['generic_name'].value_counts()
        mask = self.rx_paid['generic_name'].map(lambda x: x in opioid_all_time.index.values)
        true_opioid = self.rx_paid[mask] 
        
        opioid2_grouped = true_opioid[true_opioid['Days'] == 0].groupby(by=['id'])

        idtestlist = true_opioid[true_opioid['Days'] == 0]['id'].drop_duplicates()

        ## those commented-out codes are used to get other entry values
        # def product_sum(df):
        #     return(df['MME'].values.dot(df['PAY_DAY_SUPPLY_CNT']e.values))

        features = pd.DataFrame()
        for ID in idtestlist:
            tmp = opioid2_grouped.get_group(ID)
            output = pd.DataFrame(tmp.groupby(by='generic_name')['PAY_DAY_SUPPLY_CNT'].agg(np.nansum)).T
        #     output = output.iloc[0:1,:]
            output.index = [ID]
            # features = pd.concat([output, features], axis=1, sort=False)
            features = features.append(output, sort=False)

        features = features.fillna(0)
        
        pca = PCA(n_components = 10) 
        X10D = pca.fit_transform(features)
        
        features_matthew_generic = pd.DataFrame(X10D, index=features.index.values)
        features_matthew_generic.columns = ['generic_pc{}'.format(x) for x in features_matthew_generic.columns]
        
        self.generic_features = features_matthew_generic

In [4]:
%%time
training_class = DataPrep(training)
training_class.seperate_frame()
training_class.get_opioid_data()
training_class.LTOT()
training_class.main_feature_extraction()
training_class.generic_feature_extraction()

C:\Users\cgn31\Anaconda3\lib\site-packages\ipykernel_launcher.py:249: RuntimeWarning: All-NaN slice encountered
C:\Users\cgn31\Anaconda3\lib\site-packages\ipykernel_launcher.py:250: RuntimeWarning: Mean of empty slice


Wall time: 43min 57s


## Next we actually run the models!!!
(this will be made into a class soon)

In [35]:
final_frame = pd.concat([training_class.response_variable[['LTOT']], 
                          training_class.main_features, 
                          training_class.generic_features], 
                          axis=1, join = 'inner').dropna()

In [36]:
from sklearn.model_selection import train_test_split

X = final_frame.iloc[:,1:]
y = final_frame.iloc[:,0].map(lambda x: 1 if x == True else 0 )
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)

Random Forest:

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate

In [41]:
%%time
rf_model = RandomForestClassifier(n_estimators=300, 
                                  max_depth=9,
                                  random_state=100)

cv_results = cross_validate(rf_model, X_train, y_train, cv=3,
                            scoring= 'roc_auc',
                            return_train_score=True,
                            return_estimator =True)

print('train roc_auc: ', np.mean(cv_results['test_score']))

rf_model.fit(X_train, y_train)
prediction_on_X_test = rf_model.predict_proba(X_test)
print('test roc_auc: ', roc_auc_score(y_test, prediction_on_X_test[:,1]))

train roc_auc:  0.865347253257638
test roc_auc:  0.8796345743057674
Wall time: 19.4 s


Neural Net:

In [43]:
from sklearn.preprocessing import MinMaxScaler

In [44]:
scaler = MinMaxScaler()
scaler.fit(X)
X_scale = scaler.transform(X)
y_array = y.values
X_train, X_test_s, y_train, y_test = train_test_split(X_scale, y_array, test_size=0.33, random_state=33)
X_train_mlp, X_validation, y_train_mlp, y_validation = train_test_split(X_train, y_train, test_size=0.15, random_state=33)

In [47]:
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

Using TensorFlow backend.


In [48]:
%%time
model = Sequential()

#First Hidden Layer
model.add(Dense(128, activation='relu', kernel_initializer='random_normal', input_dim=X_train.shape[1]))

#Second  Hidden Layer
model.add(Dense(64, activation='relu', kernel_initializer='random_normal'))

# #Third  Hidden Layer
# model.add(Dense(16, activation='relu', kernel_initializer='random_normal'))

#Output Layer
model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# there's no easy way to incorporate roc_auc in MLP's training process
# so I use accuracy, which can be a proxy of roc_auc, to train models
# then use roc_auc to assess the prediction on test set

# add early stop to prevent overfitting
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
mc = ModelCheckpoint('best_model2.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

history = model.fit(X_train, y_train,
                    validation_split = 0.15,
                    epochs=4000, verbose=0, callbacks=[es, mc])

saved_model = load_model('best_model2.h5')

_, train_acc = saved_model.evaluate(X_train, y_train, verbose=0)
_, test_acc = saved_model.evaluate(X_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 00001: val_loss improved from inf to 0.45808, saving model to best_model2.h5

Epoch 00002: val_loss improved from 0.45808 to 0.44879, saving model to best_model2.h5

Epoch 00003: val_loss improved from 0.44879 to 0.44844, saving model to best_model2.h5

Epoch 00004: val_loss improved from 0.44844 to 0.44812, saving model to best_model2.h5

Epoch 00005: val_loss did not improve from 0.44812

Epoch 00006: val_loss improved from 0.44812 to 0.44430, saving model to best_model2.h5

Epoch 00007: val_loss improved from 0.44430 to 0.44394, saving model to best_model2.h5

Epoch 00008: val_loss did not improve from 0.44394

Epoch 00009: val_loss did not improve from 0.44394

Epoch 00010: val_loss improved from 0.44394 to 0.44114, saving model to best_model2.h5

Epoch 00011: val_loss did not improve from 0.44114

Epoch 00012: val_loss did not improve from 0.44114

Epoch 00013: val_loss did n


Epoch 00119: val_loss did not improve from 0.43510

Epoch 00120: val_loss did not improve from 0.43510

Epoch 00121: val_loss did not improve from 0.43510

Epoch 00122: val_loss did not improve from 0.43510

Epoch 00123: val_loss did not improve from 0.43510

Epoch 00124: val_loss did not improve from 0.43510

Epoch 00125: val_loss did not improve from 0.43510

Epoch 00126: val_loss did not improve from 0.43510

Epoch 00127: val_loss did not improve from 0.43510

Epoch 00128: val_loss did not improve from 0.43510

Epoch 00129: val_loss did not improve from 0.43510

Epoch 00130: val_loss did not improve from 0.43510

Epoch 00131: val_loss did not improve from 0.43510
Epoch 00131: early stopping
Train: 0.810, Test: 0.499
Wall time: 1min 56s


In [49]:
prediction_on_X_train = saved_model.predict_proba(X_train)
prediction_on_X_train = [x[0] for x in prediction_on_X_train]
print('roc_auc on train: ', roc_auc_score(y_train, prediction_on_X_train))

prediction_on_X_test = saved_model.predict_proba(X_test_s)
prediction_on_X_test = [x[0] for x in prediction_on_X_test]
print('roc_auc on test: ', roc_auc_score(y_test, prediction_on_X_test))

roc_auc on train:  0.8755809082094415
roc_auc on test:  0.882683602888821


Light GBM:

In [52]:
import lightgbm as lgb

In [54]:
%%time
X = final_frame.iloc[:,1:]
y = final_frame.iloc[:,0].map(lambda x: 1 if x == True else 0 )
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)

bst = lgb.LGBMClassifier(boosting_type= 'gbdt',
                        objective = 'binary',
                        max_depth = 3,
                        n_estimators = 1000)

bst.fit(X_train.values, y_train.values)

prediction_on_X_train = bst.predict_proba(X_train)
print('train roc_auc: ', roc_auc_score(y_train, prediction_on_X_train[:,1]))

prediction_on_X_test = bst.predict_proba(X_test)
print('test roc_auc: ', roc_auc_score(y_test, prediction_on_X_test[:,1]))

train roc_auc:  0.9292614434081056
test roc_auc:  0.8705631910049059
Wall time: 1.53 s


## Combined accuracy

In [55]:
NN_predict = saved_model.predict_proba(X_test_s)
bst_predict = bst.predict_proba(X_test)
rf_predict = rf_model.predict_proba(X_test)

nn_prob = np.array([ x[0] for x in NN_predict])
bst_prob = bst_predict[:,1]
rf_prob = rf_predict[:,1]

avg = (nn_prob+bst_prob+rf_prob)/3
avg

print('test roc_auc: ', roc_auc_score(y_test, avg))

test roc_auc:  0.8812656001815294


Looks like the class is working!!!